In [4]:
%load_ext autoreload
%autoreload 2
import sys,os
import pickle
from eknot_utils import EventNode,NodeParams
from utils import loadPickle,triHits,norm2,printTopWords
from eknot_utils import *

[resDocInd,tweetsObj,tweetsObjDedup,tweetsScore]= loadPickle('../output/summary_20160107_0114_80_1.pickle')
[Xs,vects,DT,ind2obj] = loadPickle('../data/20160107_0114.pickle')
rootParams,rootNodeDescriptor = loadPickle('../output/plsa_20160107_0114_80.pickle')

from time import time

In [5]:
topK = 50 # ent used
i = 1
t0 = time()
Pw_zs = rootNodeDescriptor.Pw_zs
Pe_z = Pw_zs[1][:,i]
evocab = vects[5].get_feature_names()
vocab = vects[0].get_feature_names()

ent_ind,ents = getEntInd(evocab,Pe_z,topK)

print( "entscore in "+str(time() - t0))
t0 = time()

newsObj = [ind2obj[n] for n in resDocInd]
XN,XEn,NEb,sentencesIn,sentencesInObj = getNewsContext(newsObj,ent_ind,ents,vocab)
print( "get news Context in "+str(time() - t0))
print len(newsObj),len(sentencesIn),len(set(sentencesIn))
t0 = time()

XT,XEt,TEb,tweetsIn,tweetsInObj = getTweetContext(tweetsObjDedup,ent_ind,ents,vocab)
print( "get tweet Context in "+str(time() - t0))




entscore in 0.0427861213684
get news Context in 0.576250076294
8 148 148
append in 0.127578020096
vectorize in 1.00077795982
ents append + vec in 0.226414203644
get tweet Context in 1.35718607903


In [6]:
_XEn = XEn
_XEt = XEt
_NEb = NEb
_TEb = TEb

In [8]:
"""
#####
rows = []
interestedEnts = ['tennessee_loc_|','7-eleven_org_|','odds_other_|']

for e in interestedEnts:
    rows.append(ent_ind[e])
rows = np.array(rows)
#rows = np.array([10,33,34,40])
XEn = _XEn[rows,:]
XEt = _XEt[rows,:]
NEb = _NEb[:,rows]
TEb = _TEb[:,rows]
#####
"""
print len(tweetsObjDedup),len(tweetsIn),len(set(tweetsIn))
t0 = time()

newsScore = XN.dot(Pw_zs[0][:,i])

tweetsScore = XT.dot(Pw_zs[0][:,i])
#tweetsScore = []
#for t in tweetsInObj:
#    tweetsScore.append(float(t.pop))
#tweetsScore = np.array(tweetsScore)

print( "init score in "+str(time() - t0))
t0 = time()

NE_ = XN.dot(XEn.T) # .multiply(NEb)
TE_ = XT.dot(XEt.T) #.multiply(TEb)

##### weight by pop
TE_indptr = TE_.indptr
TE_data = TE_.data
for (XTid,t) in enumerate(tweetsInObj):
    st = TE_indptr[XTid]
    en = TE_indptr[XTid+1]
    TE_data[st:en]*=float(t.pop)
#####

NE,EN = normBypartite(NE_)
TE,ET = normBypartite(TE_)
print( "graph constr in "+str(time() - t0))
t0 = time()

nScore, tScore = triHits(newsScore, tweetsScore, NE, EN, TE, ET, 0.2, 0.2, 5)
print( "trihits in "+str(time() - t0))
t0 = time()

1697 1660 1660
init score in 0.00125694274902
graph constr in 0.0322840213776
==t converge 4 4.61522380969e-08
trihits in 0.00647687911987


In [9]:
#nScore, tScore = triHits(np.ones(newsScore.shape), np.ones(tweetsScore.shape), NE, EN, TE, ET, 1, 1, 50)
print np.linalg.norm(norm2(newsScore)[:10] - nScore[:10])
print "========="
kSummary=5
printSummary(norm2(newsScore),norm2(tweetsScore),sentencesIn,sentencesInObj,tweetsIn,tweetsInObj,kSummary)
print "*****"
print "*****"
print "*****"
printSummary(nScore,tScore,sentencesIn,sentencesInObj,tweetsIn,tweetsInObj,kSummary)

0.0264605331058
76 0.191029945076 2016-01-08 06:04:16 President  Barack Obama and congressional  Democrats are ramping up calls for increased  background checks for gun purchases and to restore  federal funding for research into  gun violence. | Is it safe for Democrats to talk gun control again?
72 0.178864744612 2016-01-08 07:52:13 In the end, like so many gun policy proposals -- universal  background checks, a  federal  database of gun sales, barring people with  mental illness from buying guns, prohibiting gun ownership for domestic abusers, prohibiting gun sales to people on the terror watch list --  Obama's executive actions are supported by a strong majority of the public and of gun owners. | Obama said gun owners would support his new restrictions. He was right.
35 0.17757356118 2016-01-09 07:53:20 In State of  Union,  Obama to leave empty seat to honor gun victims,  underscore  gun control effort | In State of Union, Obama to leave empty seat to honor gun victims, underscore g

In [10]:
res1 = printRankedEnt(NE.T.dot(nScore),ents,100)
print "===="
res2 = printRankedEnt(TE.T.dot(tScore),ents,100)
print "==="
for e in ents:
    print res2[e] - res1[e],e

====
===
0 barack_obama_per_|
-4 gun_control_other_|
-10 background_check_other_|
-9 gun_violence_other_|
3 democratic_party_(united_states)_org_|
1 white_house_loc_|
-1 firearm_other_|
-20 united_states_congress_org_|
7 second_amendment_to_the_united_states_constitution_other_|
-3 mental_health_other_|
28 martin_luther_king,_jr._per_|
-1 united_states_loc_|
-1 society_other_|
1 choreography_other_|
-5 federal_government_of_the_united_states_other_|
-24 sandy_hook_elementary_school_shooting_other_|
-29 connecticut_loc_|
11 underscore_other_|
-5 michelle_obama_per_|
25 national_rifle_association_org_|
-7 hostage_other_|
-13 newtown,_connecticut_loc_|
24 new_york_city_other_|
-11 militia_other_|
-5 executive_order_other_|
28 monarch_other_|
3 president_of_the_united_states_other_|
23 art_other_|
-20 ammunition_other_|
7 soviet_union_loc_|
-38 grassroots_other_|
0 universal_health_care_other_|
-6 legislation_other_|
9 washington,_d.c._loc_|
-14 technology_other_|
13 republican_party_(unit

In [11]:
for i in range(topK):
#i=18
    print ents[i]
    printTopWords(i,XEn,vocab,10)
    printTopWords(i,XEt,vocab,10)
    print "+++++++++"

barack_obama_per_|
[u'obama', u'gun', u'guns', u'background', u'president', u'control', u'executive', u'checks', u'violence', u'malloy']
[u'obama', u'gun', u'new', u'control', u'support', u'seat', u'state', u'union', u'leave', u'owners']
+++++++++
gun_control_other_|
[u'gun', u'obama', u'control', u'laws', u'rights', u'president', u'activists', u'measures', u'efforts', u'congress']
[u'gun', u'control', u'obama', u'talk', u'democrats', u'seat', u'victims', u'leave', u'honor', u'underscore']
+++++++++
background_check_other_|
[u'gun', u'background', u'checks', u'obama', u'executive', u'guns', u'federal', u'purchases', u'fbi', u'firearms']
[u'background', u'gun', u'wait', u'boy', u'buying', u'check', u'checks', u'control', u'gop', u'new']
+++++++++
gun_violence_other_|
[u'gun', u'violence', u'obama', u'president', u'house', u'seat', u'americans', u'supporters', u'tuesday', u'rights']
[u'gun', u'violence', u'seat', u'white', u'leave', u'house', u'victims', u'lady', u'sotu', u'union']
+++++